# **Invoice Data Analysis**
### **Author: Davit Davtyan** 
#### **Task Description:** 

There is a binary(pickled) file invoices_new.pkl, which contains information of invoices. You may assume that the information has been extracted by scanning paper invoices using OCR. Each invoice has its own properties (id, name, creation date etc.) and a list of invoice items. Like invoice, invoice items also have their own properties.
Tere is also a file containing a list of IDs of expired invoices - expired_invoices.txt

Write a `DataExtractor class` that has the functionality to load the dataset and transform the unstructured data into a flat data with the following columns:
- invoice_id: int
- created_on: datetime64[ns]
- invoiceitem_id: int
- invoiceitem_name: str
- type: str (use this conversion table: {0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'})
- unit_price: int
- total_price: int (unit_price*quantity)
- percentage_in_invoice: float (unit_price*quantity / invoice_total)
- is_expired: bool (whether invoice_id is contained expired_invoices.txt or not)

Make sure that the columns in the resulting dataframe are of the correct type, as described above. The final dataframe must be sorted by invoice_id and invoiceitem_id in the ascending order.

#### **Import Necessary Libraries**

In [1]:
import os
import pandas as pd
import pickle
import re
from num2words import num2words

#### **Load Data**

In [2]:
with open('data/invoices_new.pkl', 'rb') as f:
    invoices_data = pickle.load(f)

print("Invoices Data:")
print(f"Number of invoices: {len(invoices_data)}")
print("Sample invoice data:")
invoices_data[0]

Invoices Data:
Number of invoices: 100
Sample invoice data:


{'id': '365371O',
 'created_on': '2019-05-27',
 'items': [{'item': {'id': 142446,
    'name': 'ii_142446',
    'unit_price': 182,
    'type': 1},
   'quantity': 6},
  {'item': {'id': 154019, 'name': 'ii_154019', 'unit_price': 176, 'type': 1},
   'quantity': 1}]}

In [3]:
with open('data/expired_invoices.txt', 'r') as f:
    expired_invoice_ids = f.read().split(', ')

expired_invoice_ids = list(map(int, expired_invoice_ids))

print(f"\nNumber of expired invoices: {len(expired_invoice_ids)}")
print("Expired Invoice IDs:")
print(expired_invoice_ids)


Number of expired invoices: 50
Expired Invoice IDs:
[305869, 377307, 391273, 385290, 331902, 397723, 343695, 340601, 347510, 325156, 379387, 348894, 322229, 326649, 377960, 325063, 369378, 319405, 383681, 367288, 356552, 394428, 381476, 381457, 349879, 343254, 366751, 330931, 363263, 326452, 379687, 356532, 383235, 313012, 368913, 352442, 315960, 351096, 331193, 392657, 352391, 389528, 338547, 379961, 337140, 323231, 371205, 378746, 307175, 340299]


#### **The Creation of DataExtractor Class**

In [4]:
class DataExtractor:
    def __init__(self, invoices_file, expired_invoices_file):
        self.invoices_file = invoices_file
        self.expired_invoices_file = expired_invoices_file
        self.invoices_data = self.load_invoices_data()
        self.expired_invoice_ids = self.load_expired_invoices()
        self.type_conversion = {'0': 'Material', 'O': 'Material', 0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'}
        self.quantity_map = self.create_quantity_map()

    def load_invoices_data(self):
        with open(self.invoices_file, 'rb') as f:
            return pickle.load(f)

    def load_expired_invoices(self):
        with open(self.expired_invoices_file, 'r') as f:
            return list(map(int, f.read().split(', ')))

    def create_quantity_map(self):
        quantity_map = {}
        for i in range(1, 101):
            quantity_map[num2words(i)] = i
        return quantity_map

    def parse_quantity(self, quantity):
        if isinstance(quantity, int):
            return quantity
        quantity = quantity.lower()
        return self.quantity_map.get(quantity, 1)

    def transform(self):
        flat_data = []

        for invoice in self.invoices_data:
            invoice_id = invoice['id']
            if isinstance(invoice_id, str):
                invoice_id = int(invoice_id[:-1])  # Extract integer part if invoice_id is a string
            created_on = pd.to_datetime(invoice['created_on'], errors='coerce')
            is_expired = invoice_id in self.expired_invoice_ids

            if 'items' not in invoice:
                continue

            invoice_total = sum(self.parse_quantity(item['quantity']) * item['item']['unit_price'] for item in invoice['items'])

            for item in invoice['items']:
                invoiceitem_id = item['item']['id']
                invoiceitem_name = item['item']['name']
                unit_price = item['item']['unit_price']
                quantity = self.parse_quantity(item['quantity'])
                total_price = unit_price * quantity
                percentage_in_invoice = total_price / invoice_total
                item_type = self.type_conversion.get(item['item']['type'], 'Other')

                flat_data.append({
                    'invoice_id': invoice_id,
                    'created_on': created_on,
                    'invoiceitem_id': invoiceitem_id,
                    'invoiceitem_name': invoiceitem_name,
                    'type': item_type,
                    'unit_price': unit_price,
                    'total_price': total_price,
                    'percentage_in_invoice': percentage_in_invoice,
                    'is_expired': is_expired
                })

        df = pd.DataFrame(flat_data)

        # Convert columns to the correct types
        df['invoice_id'] = df['invoice_id'].astype(int)
        df['created_on'] = pd.to_datetime(df['created_on'], errors='coerce')
        df['invoiceitem_id'] = df['invoiceitem_id'].astype(int)
        df['invoiceitem_name'] = df['invoiceitem_name'].astype(str)
        df['type'] = df['type'].astype(str)
        df['unit_price'] = df['unit_price'].astype(int)
        df['total_price'] = df['total_price'].astype(int)
        df['percentage_in_invoice'] = df['percentage_in_invoice'].astype(float)
        df['is_expired'] = df['is_expired'].astype(bool)

        # Drop rows with invalid dates
        df = df.dropna(subset=['created_on'])

        # Sort by invoice_id and invoiceitem_id in ascending order
        df = df.sort_values(by=['invoice_id', 'invoiceitem_id']).reset_index(drop=True)

        return df
    
    def save_to_csv(self, df, output_file):
        df.to_csv(output_file, index=False)

### **Usage**

In [5]:
extractor = DataExtractor('data/invoices_new.pkl', 'data/expired_invoices.txt')
flat_df = extractor.transform()
extractor.save_to_csv(flat_df, 'transformed_invoices.csv')

flat_df.head()

,invoice_id,created_on,invoiceitem_id,invoiceitem_name,type,unit_price,total_price,percentage_in_invoice,is_expired
0,301695,2019-04-26,103215,ii_103215,Other,135,945,0.374851,False
1,301695,2019-04-26,166227,ii_166227,Equipment,118,708,0.280841,False
2,301695,2019-04-26,171394,ii_171394,Other,128,128,0.050774,False
3,301695,2019-04-26,195625,ii_195625,Other,148,740,0.293534,False
4,304245,2019-03-17,121446,ii_121446,Service,158,632,0.128429,False


In [6]:
flat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486 entries, 0 to 485
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   invoice_id             486 non-null    int32         
 1   created_on             486 non-null    datetime64[ns]
 2   invoiceitem_id         486 non-null    int32         
 3   invoiceitem_name       486 non-null    object        
 4   type                   486 non-null    object        
 5   unit_price             486 non-null    int32         
 6   total_price            486 non-null    int32         
 7   percentage_in_invoice  486 non-null    float64       
 8   is_expired             486 non-null    bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int32(4), object(2)
memory usage: 23.4+ KB


#### **END**